# National Model 3b
for Custom Model

- Equifax, TU, Experian National model 
- Features selection on national data
- Updated time split on Equifax
    - Training: 2016-04-01 to 2020-12-31
    - Valid: 2021-01-01 to 2021-03-31


The notebook covers all three steps for creatign a new national model; feature selection, validation model and fold-in model. depending on the applied use case, not each step may be necessary (e.g., if the feature selection does not change compaed to the previous interation, step 1 may be skipped, etc.

In [1]:
import pandas as pd
import numpy as np

import model_engine
model_engine.__version__

'v1.13.0'

In [2]:
from model_engine.power.post_sale import NationalModelBuilder
from model_engine.io.loaders import load_json

import json

/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/zamlexplain/utils_compile.py:23: NumbaPendingDeprecationWarning: The 'pycc' module is pending deprecation. Replacement technology is being developed.

Pending Deprecation in Numba 0.57.0. For more information please see: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-the-numba-pycc-module
  from numba.pycc import CC
/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/lyt/.conda/envs/penfed_modeling/lib/python3.1

In [3]:
import os
import numba as nb
n_jobs = os.getenv('CPU_LIMIT', default=-1)
n_jobs = 20
print(n_jobs)
nb.set_num_threads(n_jobs)

20


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
input_client_name = 'penfed'
input_project_name = 'autoloan'
input_bureau_name = 'normalized'
model_type = 'standard_model_member_data' # options are 'standard_model', 'mega_model', 'standard_model_member_data', 'standard_model_ln_data', ''- None defaults to 'standard_model'

print(input_client_name, input_project_name, input_bureau_name, model_type)

# national_model_id = 'nationalmodel1b'

penfed autoloan normalized standard_model_member_data


## Create a national model

In [6]:
# client' feature selection model

feature_selection_model_id = 'nationalmodel3a'

In [7]:
feature_selection_model_output_path = f'/d/shared/users/lyt/penfed_custom_model_national_models/{feature_selection_model_id}'
feature_selection_model_output_path

'/d/shared/users/lyt/penfed_custom_model_national_models/nationalmodel3a'

In [8]:
config = {
  "model_type": "standard_model",
  "storage_location": "s3",
  "lookalike": {
    "bureau": "normalized",
    "accounts":[
      "auto"
    ],
    "splits": [
      "train",
      "valid",
      "test"
    ],
    "pull_names": None,
    "states": None,
    "max_row": None,
    "tables": ['trade', 'inq', 'bnkr', 'collec', 'target', 'app', 'member'],
  },
  "model_config": {
      "data_split": {
          "train": {
            "start_date": "2016-04-01",
            "end_date": "2021-01-01"
          },
          "valid": {
            "start_date": "2021-01-01",
            "end_date": "2021-04-01"
          },
          "test": {
            "start_date": "2021-04-01",
            "end_date": "2022-01-01"
          }
    },
    "target": "final_DQ60_m24",
    "base_features": f'{feature_selection_model_output_path}/keep_features.json',
#    "feature_filter_rules": "/home/lyt/CODEBASE/model-engine_penfed/model-engine/model_engine/assets/power/feature_rules/intuitive_features.json",
    "memory_efficient": True,
    "pipeline_factory": None,
    "bivariate_fe_instructions": [],
    "monotonic_constraints_list": [],
    "exclusion_list": [],
    "feature_definition_list": [],
    "key_factor_mapping_list": [],
    "fold_valid": True,
    "mandatory_features": []
  }
}

In [9]:
national_model_id = 'nationalmodel3b'
national_model_output_path = f'/d/shared/users/lyt/penfed_custom_model_national_models/{national_model_id}'


In [10]:
national_model_output_path

'/d/shared/users/lyt/penfed_custom_model_national_models/nationalmodel3b'

In [11]:
nmb = NationalModelBuilder(
    configuration=config,
    model_output_path=national_model_output_path,
    overwrite_model_output=True
)
nmb.run()

Attempting to create a national model with full data split here /d/shared/users/lyt/penfed_custom_model_national_models/nationalmodel3b/supporting_model
Sampling experian, national_1.3
Sampling experian, national_2
Sampling equifax, national_1.1
Sampling equifax, national_2
Sampling equifax, national_1_home_equity
Sampling equifax, national_3
Sampling transunion, national_3_refresh
Sampling transunion, national_4
Sampling experian, national_1.3
Sampling experian, national_2
Sampling equifax, national_1.1
Sampling equifax, national_2
Sampling equifax, national_1_home_equity
Sampling equifax, national_3
Sampling transunion, national_3_refresh
Sampling transunion, national_4
Sampling experian, national_1.3
Sampling experian, national_2
Sampling equifax, national_1.1
Sampling equifax, national_2
Sampling equifax, national_1_home_equity
Sampling equifax, national_3
Sampling transunion, national_3_refresh
Sampling transunion, national_4



/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/fsspec/registry.py:290: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/fsspec/registry.py:290: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/fsspec/registry.py:290: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify

parsing model-builder asset








/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/fsspec/registry.py:290: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/fsspec/registry.py:290: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/fsspec/registry.py:290: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should 

Configuring model builder


...


INFO:zaml.artifact_engine.logger:Finished <train_fe_data>, total time spent: 0:10:18.497135
INFO:zaml.artifact_engine.logger:Executing FeDataArtifact <test_fe_data>...


-------------------------
Name: FillNA
Transformer type: FillNA
Number of features: 290
Time spent: 82.109s
-------------------------
Name: GeneralFeatureSelection
Transformer type: GeneralFeatureSelection
Number of features: 290
Time spent: 0.001s
-------------------------
Name: app
Transformer type: None
Number of features: 18
Time spent: 0.000s
-------------------------
Name: bnkr
Transformer type: None
Number of features: 8
Time spent: 0.000s
-------------------------
Name: collec
Transformer type: None
Number of features: 5
Time spent: 0.000s
-------------------------
Name: inq
Transformer type: None
Number of features: 13
Time spent: 0.000s
-------------------------
Name: member
Transformer type: None
Number of features: 28
Time spent: 0.000s
-------------------------
Name: trade
Transformer type: None
Number of features: 236
Time spent: 0.000s
-------------------------
Name: app FE
Transformer type: OneToOneEngine
Number of features: 0
Time spent: 0.072s
------------------------


INFO:zaml.artifact_engine.logger:Finished <test_fe_data>, total time spent: 0:00:26.450873
INFO:zaml.artifact_engine.logger:Executing StaticAssetArtifact <static_asset>...


-------------------------
Name: FillNA
Transformer type: FillNA
Number of features: 290
Time spent: 6.380s
-------------------------
Name: GeneralFeatureSelection
Transformer type: GeneralFeatureSelection
Number of features: 290
Time spent: 0.001s


INFO:zaml.artifact_engine.logger:Finished <static_asset>, total time spent: 0:00:01.934707
INFO:zaml.artifact_engine.logger:Executing TrainHistoryArtifact <train_history>...
INFO:zaml.artifact_engine.logger:Finished <train_history>, total time spent: 0:00:00.000577
INFO:zaml.artifact_engine.logger:Executing BestModelParamsArtifact <best_model_params>...
INFO:zaml.artifact_engine.logger:Finished <best_model_params>, total time spent: 0:00:00.000584
INFO:zaml.artifact_engine.logger:Executing ScoresArtifact <train_scores>...
INFO:zaml.artifact_engine.logger:Finished <train_scores>, total time spent: 0:02:43.351586
INFO:zaml.artifact_engine.logger:Executing SubmodelScoresArtifact <train_submodel_scores>...
INFO:zaml.artifact_engine.logger:Finished <train_submodel_scores>, total time spent: 0:00:00.000742
INFO:zaml.artifact_engine.logger:Executing ScoresArtifact <test_scores>...
INFO:zaml.artifact_engine.logger:Finished <test_scores>, total time spent: 0:00:08.595917
INFO:zaml.artifact_engi

In [14]:
national_model_id

'nationalmodel3b'

In [15]:
ftr_importance = pd.read_parquet(f'/d/shared/users/lyt/penfed_custom_model_national_models/{national_model_id}/feature_importance.parquet')
ftr_importance.head(10)

,feature,importance
0,trade_blnc_to_hc_accts_never_dq__max_by_acctTy...,0.009496
1,inq_inquiries__sum,0.008161
2,trade_blncAmt__mean_by_acctType_auto,0.006888
3,trade_acctType__count,0.006750
4,trade_hcAmt_all_non_derog_accts__sum,0.006726
5,trade_months_since_openDate__mean_by_acctType_...,0.006494
6,trade_blnc_to_hc_all_non_derog_accts__max_by_a...,0.006096
7,trade_clAmt__mean_by_prtfType_revolving,0.005915
8,inq_inquiries__count_by_indCode_auto_mortgage,0.005705
9,trade_months_since_rptDate__max_by_acctType_cr...,0.005532
